# Gallery Example: Finite Repairmen Model

This is a classic finite population model representing a repairmen system:
- **Machines**: Finite number of machines that can break down
- **Repairmen**: Limited number of servers to fix machines
- **Topology**: Machines cycle between working (delay) and being repaired (queue)
- **Population**: Finite and fixed number of jobs (machines)

This model is fundamental for:
- Manufacturing systems
- Computer systems reliability
- Any finite population service system

In [ ]:
from line_solver import *
import numpy as np
import random
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_repairmen(nservers=1, seed=23000):    """Create finite repairmen model        Parameters:    - nservers: Number of repair servers (default 1)    - seed: Random seed for reproducibility (default 23000)    """    model = Network('Finite repairmen CQN')        # Set random seed    random.seed(seed)    np.random.seed(seed)        # Block 1: nodes    repair_queue = Queue(model, 'Queue1', SchedStrategy.PS)    repair_queue.set_number_of_servers(nservers)        working_delay = Delay(model, 'Delay1')  # Machines working (operational time)        # Block 2: classes    # Random population of machines (between 3 and 13)    population = int(random.random() * 10 + 3)    jobclass = ClosedClass(model, 'Class1', population, repair_queue, 0)        # Set service times    repair_rate = random.random() + 1  # Random repair rate between 1 and 2    repair_queue.set_service(jobclass, Exp.fit_mean(repair_rate))        working_delay.set_service(jobclass, Exp.fit_mean(2.0))  # Mean time between failures        # Block 3: topology - simple cycle    P = model.init_routing_matrix()    P.add_route(jobclass, repair_queue, working_delay, 1.0)  # Repaired -> Working    P.add_route(jobclass, working_delay, repair_queue, 1.0)  # Broken -> Repair        model.link(P)        return model, population, nservers# Create the modelmodel, population, nservers = gallery_repairmen(nservers=2, seed=23000)print(f"Machines: {population}")print(f"Repairmen: {nservers}")

## Model Interpretation

- **Queue1**: Repair facility where broken machines wait to be fixed
- **Delay1**: Operational state where machines work until they break down
- **Population**: Fixed number of machines in the system
- **Servers**: Number of repair technicians available

Key metrics:
- **Queue length at repair**: Average number of machines waiting/being repaired
- **Queue length at delay**: Average number of operational machines  
- **Utilization**: Fraction of time repair facility is busy
- **Throughput**: Rate of completions (repairs per time unit)

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver (exact for closed models)
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver 
solver_ctmc = CTMC(model)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# NC Solver
solver_nc = NC(model)
avg_table_nc = solver_nc.avg_table()
print("\nNC Solver:")
print(avg_table_nc)